Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [22]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [23]:
pickle_file = '/home/garfield/tmp/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [24]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28, 1) (20000, 10)
Validation set (1000, 28, 28, 1) (1000, 10)
Test set (1000, 28, 28, 1) (1000, 10)


In [25]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [36]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(data):
    print('---------------------------')      
    print('data', data)
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print('conv', conv)
    hidden = tf.nn.relu(conv + layer1_biases)
    print('hidden', hidden)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print('conv', conv)
    hidden = tf.nn.relu(conv + layer2_biases)
    print('hidden', hidden)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape', reshape)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden', hidden)
    out= tf.matmul(hidden, layer4_weights) + layer4_biases
    print('out', out)
    return out
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

---------------------------
data Tensor("Placeholder:0", shape=(16, 28, 28, 1), dtype=float32)
conv Tensor("Conv2D:0", shape=(16, 14, 14, 16), dtype=float32)
hidden Tensor("Relu:0", shape=(16, 14, 14, 16), dtype=float32)
conv Tensor("Conv2D_1:0", shape=(16, 7, 7, 16), dtype=float32)
hidden Tensor("Relu_1:0", shape=(16, 7, 7, 16), dtype=float32)
reshape Tensor("Reshape:0", shape=(16, 784), dtype=float32)
hidden Tensor("Relu_2:0", shape=(16, 64), dtype=float32)
out Tensor("add_3:0", shape=(16, 10), dtype=float32)
---------------------------
data Tensor("Const:0", shape=(1000, 28, 28, 1), dtype=float32)
conv Tensor("Conv2D_2:0", shape=(1000, 14, 14, 16), dtype=float32)
hidden Tensor("Relu_3:0", shape=(1000, 14, 14, 16), dtype=float32)
conv Tensor("Conv2D_3:0", shape=(1000, 7, 7, 16), dtype=float32)
hidden Tensor("Relu_4:0", shape=(1000, 7, 7, 16), dtype=float32)
reshape Tensor("Reshape_4:0", shape=(1000, 784), dtype=float32)
hidden Tensor("Relu_5:0", shape=(1000, 64), dtype=float32)
out T

In [28]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 1000: 0.749479
Minibatch accuracy: 81.2%
Validation accuracy: 84.8%
Test accuracy: 89.5%


Minibatch loss at step 950: 0.605082
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%


Minibatch loss at step 900: 0.653404
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%


Minibatch loss at step 850: 0.782422
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%


Minibatch loss at step 800: 1.011311
Minibatch accuracy: 62.5%
Validation accuracy: 84.2%


...

Minibatch loss at step 200: 1.017870
Minibatch accuracy: 68.8%
Validation accuracy: 77.8%


Minibatch loss at step 150: 0.664678
Minibatch accuracy: 75.0%
Validation accuracy: 71.0%


Minibatch loss at step 100: 1.229475
Minibatch accuracy: 62.5%
Validation accuracy: 60.1%


Minibatch loss at step 50: 2.045574
Minibatch accuracy: 25.0%
Validation accuracy: 18.3%


Initialized
Minibatch loss at step 0: 3.115915
Minibatch accuracy: 0.0%
Validation accuracy: 9.6%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---